# Plotly

Plotly to bardzo rozbudowana biblioteka do tworzenia interaktywnych wykresów. Najbardziej podobną do `pyplot`'a konstrukcję ma `plotly.express` który poznamy na tych zajęciach.

In [ ]:
import plotly.express as px
from gapminder import gapminder
import numpy as np
import pandas as pd

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

## gapminder

In [ ]:
gapminder

In [ ]:
gapminder_2007 = gapminder[gapminder['year'] == 2007]

fig = px.scatter(
    data_frame=gapminder_2007,
    x='gdpPercap',
    y='lifeExp',
    color='continent',
    custom_data=['country', 'pop'],
    size='pop',
    size_max=55,
    log_x=True,
    labels={
        "lifeExp": 'Life expectancy (years)',
        "gdpPercap": 'GDP per capita (USD)'
    }
)

hovertemplate = "<b>%{customdata[0]}</b><br>" +\
                "GDP per Capita: %{x:$,.2f}<br>" +\
                "Life Expectation: %{y}<br>" +\
                "Population: %{customdata[1]:,.0f}"

fig.update_traces(
    hovertemplate=hovertemplate
)


fig.show()

In [ ]:
# co jak chcemy sprawdzić dla różnych lat?

fig = px.scatter(
    data_frame=gapminder,
    x='gdpPercap',
    y='lifeExp',
    color='continent',
    animation_frame="year",
    animation_group="country",
    custom_data=['country', 'pop'],
    size='pop',
    size_max=55,
    log_x=True,
    labels={
        "lifeExp": 'Life expectancy (years)',
        "gdpPercap": 'GDP per capita (USD)'
    },
    range_x=[100,100000], 
    range_y=[25,90]
)


fig["layout"].pop("updatemenus")

fig.update_traces(
    hovertemplate=hovertemplate
)


fig.show()

In [ ]:
# Jak się zmieniał rozkład w latach?

fig = px.scatter(
    data_frame=gapminder,
    x='gdpPercap',
    y='lifeExp',
    color='continent',
    animation_frame="year",
    animation_group="country",
    custom_data=['country', 'pop'],
    size='pop',
    size_max=55,
    log_x=True,
    labels={
        "lifeExp": 'Life expectancy (years)',
        "gdpPercap": 'GDP per capita (USD)'
    },
    range_x=[100,100000], 
    range_y=[25,90],
    marginal_y="box"
)


fig["layout"].pop("updatemenus")

fig.update_traces(
    hovertemplate=hovertemplate
)


fig.show()

In [ ]:
# lifeExp w różnych krajach w kolejnych latach (wykres liniowy) – interaktywnie?

In [ ]:
countries = gapminder['country'].unique()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id="dropdown",
        options=[
            {
                "label": c,
                "value":c
            } for c in countries],
        value=['China', 'Poland', 'United States'], # wartości początkowe
        multi=True # dopuszczamy umieszczenie wielu krajów na wykresie
    ),
    dcc.Graph(id="linechart"),
])

@app.callback(
    Output("linechart", "figure"), 
    [Input("dropdown", "value")])
def update_bar_chart(selected):
    df = gapminder[gapminder['country'].isin(selected)]
    fig = px.line(
        df, x="year", y="lifeExp", color='country')
    return fig

app.run_server(debug=True, use_reloader=False)

In [ ]:
# Rozkład popularności imion dzieci w Polsce

In [ ]:
DATA_URL = 'https://api.dane.gov.pl/media/resources/20190822/IMIONA_ur_01.01.2019-30.06.2019.xlsx'

names = pd.read_excel(DATA_URL)

In [ ]:
names

In [ ]:
most_popular = names.sort_values('Liczba', ascending=False)


app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Slider(
        id='slider',
        min=5,
        max=200,
        step=1,
        value=10,
    ),
    dcc.Dropdown(
        id='dropdown',
        options=[
            {
                "label": x,
                "value":x
            } for x in ['M', 'K']],
        value=['M', 'K'], # wartości początkowe
        multi=True # dopuszczamy umieszczenie wielu krajów na wykresie
    ),
    dcc.Graph(id="barchart")
])


@app.callback(
    Output('barchart', 'figure'),
    [Input('slider', 'value'), Input('dropdown', 'value')])
def update_bar_chart(n_names, genders):
    df = most_popular[most_popular['Płeć'].isin(genders)][:n_names]
    fig = px.bar(
        df, x="Imię", y="Liczba")
    return fig


app.run_server(debug=True, use_reloader=False)